In [33]:
import backtrader as bt
from datetime import datetime
import yfinance as yf

class MovingAverageCrossover(bt.Strategy):
    params = (('fast_length', 10), ('slow_length', 30),)

    def __init__(self):
        self.data_close = self.datas[0].close
        
        # The moving averages
        self.fast_ma = bt.indicators.SimpleMovingAverage(self.datas[0], period=self.params.fast_length)
        self.slow_ma = bt.indicators.SimpleMovingAverage(self.datas[0], period=self.params.slow_length)

    def next(self):
        if self.fast_ma[0] > self.slow_ma[0] and self.fast_ma[-1] <= self.slow_ma[-1]:
            # print(f"Buy signal on {self.data.datetime.date()}")
            self.buy()
        elif self.fast_ma[0] < self.slow_ma[0] and self.fast_ma[-1] >= self.slow_ma[-1]:
            # print(f"Sell signal on {self.data.datetime.date()}")
            self.sell()

class RSI_Bollinger(bt.Strategy):
    params = (('rsi_length', 14), ('bollinger_length', 20), ('bollinger_dev', 2),)

    def __init__(self):
        self.data_close = self.datas[0].close
        self.rsi = bt.indicators.RSI(self.datas[0], period=self.params.rsi_length)
        self.bollinger = bt.indicators.BollingerBands(self.datas[0], period=self.params.bollinger_length, devfactor=self.params.bollinger_dev)

    def next(self):
        if self.rsi < 30 and self.data_close < self.bollinger.lines.bot:
            print(f"Buy signal on {self.data.datetime.date()},{self.data.datetime.time()} with RSI: {self.rsi[0]}, price: {self.data_close[0]}, lower band: {self.bollinger.lines.bot[0]}")
            self.buy()
        elif self.rsi > 70 and self.data_close > self.bollinger.lines.top:
            print(f"Sell signal on {self.data.datetime.date()},{self.data.datetime.time()} with RSI: {self.rsi[0]}, price: {self.data_close[0]}, upper band: {self.bollinger.lines.top[0]}")
            self.sell()
        self.last_order_time = self.data.datetime


# Set up the data and engine
cerebro = bt.Cerebro()
cerebro.addstrategy(RSI_Bollinger)

# Set initial conditions
cerebro.broker.set_cash(100_000)
cerebro.broker.setcommission(commission=0.001)

# Add observers and analyzers
cerebro.addobserver(bt.observers.Broker)
cerebro.addobserver(bt.observers.DrawDown)
cerebro.addanalyzer(bt.analyzers.SharpeRatio, _name='sharpe_ratio')
cerebro.addanalyzer(bt.analyzers.Transactions, _name='transactions')
cerebro.addanalyzer(bt.analyzers.TradeAnalyzer, _name='trade_analyzer')

# Get data
data = bt.feeds.PandasData(dataname=yf.download('ITC.NS', interval='1m', period='5d'))
cerebro.adddata(data)

# Run the strategy
results = cerebro.run()
strategy = results[0]

# Print results
print('Final Portfolio Value: %.2f' % cerebro.broker.getvalue())
print('Sharpe Ratio:', strategy.analyzers.sharpe_ratio.get_analysis())
print('Transactions:', strategy.analyzers.transactions.get_analysis())
print('Trade Analysis Results:', strategy.analyzers.trade_analyzer.get_analysis())

[*********************100%%**********************]  1 of 1 completed


Buy signal on 2024-06-10,04:29:00 with RSI: 29.887809324483413, price: 436.5, lower band: 436.83078224814426
Sell signal on 2024-06-10,07:42:00 with RSI: 71.98700404907036, price: 439.0, upper band: 438.7619088331895
Sell signal on 2024-06-10,07:44:00 with RSI: 71.19723593576892, price: 439.1499938964844, upper band: 439.0013888698637
Buy signal on 2024-06-10,08:54:00 with RSI: 26.80471012946633, price: 437.95001220703125, lower band: 437.9861295744017
Buy signal on 2024-06-10,08:59:00 with RSI: 20.544538118648234, price: 437.25, lower band: 437.4385566396578
Buy signal on 2024-06-11,03:45:00 with RSI: 23.678164064026305, price: 434.8500061035156, lower band: 435.7901683643598
Buy signal on 2024-06-11,03:49:00 with RSI: 29.93020263305607, price: 434.5, lower band: 434.8533283875967
Buy signal on 2024-06-11,06:34:00 with RSI: 28.25734093530191, price: 434.54998779296875, lower band: 434.9528255169561
Buy signal on 2024-06-11,06:54:00 with RSI: 29.316087810923918, price: 434.299987792968

In [47]:
import backtrader as bt
from datetime import datetime
import yfinance as yf

class RSI_Bollinger(bt.Strategy):
    params = (('rsi_length', 14), ('bollinger_length', 20), ('bollinger_dev', 2),)

    def __init__(self):
        self.data_close = self.datas[0].close
        self.rsi = bt.indicators.RSI(self.datas[0], period=self.params.rsi_length)
        self.bollinger = bt.indicators.BollingerBands(self.datas[0], period=self.params.bollinger_length, devfactor=self.params.bollinger_dev)
        self.order = None  # To keep track of pending orders

    def log(self, txt, dt=None):
        ''' Logging function for this strategy'''
        dt = dt or self.datas[0].datetime.datetime(0)
        print(f'{dt.isoformat()} - {txt}')

    def next(self):
        if self.order:
            return  # Skip if there's a pending order

        if self.rsi < 30 and self.data_close < self.bollinger.lines.bot:
            self.log(f"Buy signal with RSI: {self.rsi[0]}, Price: {self.data_close[0]}, Lower Band: {self.bollinger.lines.bot[0]}")
            self.order = self.buy()
        elif self.rsi > 70 and self.data_close > self.bollinger.lines.top:
            self.log(f"Sell signal with RSI: {self.rsi[0]}, Price: {self.data_close[0]}, Upper Band: {self.bollinger.lines.top[0]}")
            self.order = self.sell()

    def notify_order(self, order):
        if order.status in [order.Submitted, order.Accepted]:
            return  # Pending order - do nothing

        if order.status in [order.Completed]:
            if order.isbuy():
                self.log(f'BUY EXECUTED, Price: {order.executed.price}, Cost: {order.executed.value}, Comm: {order.executed.comm}')
            elif order.issell():
                self.log(f'SELL EXECUTED, Price: {order.executed.price}, Cost: {order.executed.value}, Comm: {order.executed.comm}')
            self.bar_executed = len(self)
        elif order.status in [order.Canceled, order.Margin, order.Rejected]:
            self.log('Order Canceled/Margin/Rejected')

        self.order = None  # Reset orders

    def notify_trade(self, trade):
        if not trade.isclosed:
            return

        self.log(f'OPERATION PROFIT, GROSS: {trade.pnl}, NET: {trade.pnlcomm}')

# Set up the data and engine
cerebro = bt.Cerebro()
cerebro.addstrategy(RSI_Bollinger)

# Set initial conditions
cerebro.broker.set_cash(100_000)
cerebro.broker.setcommission(commission=0.001)

# Add observers and analyzers
cerebro.addobserver(bt.observers.Broker)
cerebro.addobserver(bt.observers.DrawDown)
cerebro.addanalyzer(bt.analyzers.SharpeRatio, _name='sharpe_ratio')
cerebro.addanalyzer(bt.analyzers.Transactions, _name='transactions')
cerebro.addanalyzer(bt.analyzers.TradeAnalyzer, _name='trade_analyzer')

data = bt.feeds.PandasData(dataname=yf.download('MARUTI.NS', interval='1m', period='5d'))
cerebro.adddata(data)

# Run the strategy
results = cerebro.run()
strategy = results[0]

# Print results
print('Final Portfolio Value: %.2f' % cerebro.broker.getvalue())
print('Sharpe Ratio:', strategy.analyzers.sharpe_ratio.get_analysis())
print('Transactions:', strategy.analyzers.transactions.get_analysis())
print('Trade Analysis Results:', strategy.analyzers.trade_analyzer.get_analysis())

[*********************100%%**********************]  1 of 1 completed


2024-06-10T04:20:00 - Buy signal with RSI: 29.481777567563583, Price: 12755.849609375, Lower Band: 12757.417677747
2024-06-10T04:21:00 - BUY EXECUTED, Price: 12755.7998046875, Cost: 12755.7998046875, Comm: 12.7557998046875
2024-06-10T06:39:00 - Buy signal with RSI: 26.607837588956755, Price: 12695.5498046875, Lower Band: 12701.40457348489
2024-06-10T06:40:00 - BUY EXECUTED, Price: 12700.0, Cost: 12700.0, Comm: 12.700000000000001
2024-06-10T07:01:00 - Buy signal with RSI: 26.137539055615292, Price: 12683.900390625, Lower Band: 12702.617677818245
2024-06-10T07:02:00 - BUY EXECUTED, Price: 12683.900390625, Cost: 12683.900390625, Comm: 12.683900390625
2024-06-10T07:02:00 - Buy signal with RSI: 28.236495798998178, Price: 12686.150390625, Lower Band: 12695.447793525702
2024-06-10T07:03:00 - BUY EXECUTED, Price: 12686.150390625, Cost: 12686.150390625, Comm: 12.686150390625
2024-06-10T07:03:00 - Buy signal with RSI: 27.45216558733982, Price: 12684.0498046875, Lower Band: 12689.073466212716
202

In [37]:
import backtrader as bt
from datetime import datetime, timedelta
import yfinance as yf
from colorama import Fore, Style

class RSI_Bollinger(bt.Strategy):
    params = (('rsi_length', 14), ('bollinger_length', 20), ('bollinger_dev', 2),)

    def __init__(self):
        self.data_close = self.datas[0].close
        self.rsi = bt.indicators.RSI(self.datas[0], period=self.params.rsi_length)
        self.bollinger = bt.indicators.BollingerBands(self.datas[0], period=self.params.bollinger_length, devfactor=self.params.bollinger_dev)
        self.order = None  # To keep track of pending orders
        self.last_order_time = None  # To track the time of the last order
        self.last_order = None  # To track the last order

    def log(self, txt, dt=None):
        ''' Logging function for this strategy'''
        dt = dt or self.datas[0].datetime.datetime(0)
        print(f'{dt.isoformat()} - {txt}')

    def next(self):
        current_time = self.datas[0].datetime.datetime(0)
        
        # Check if 30 minutes have passed since the last order
        if self.last_order_time and current_time < self.last_order_time + timedelta(minutes=30):
            return

        if self.order:
            return  # Skip if there's a pending order

        if self.rsi < 30 and self.data_close < self.bollinger.lines.bot and self.last_order != 'buy':
            self.log(Fore.LIGHTGREEN_EX + f"Buy signal with RSI: {self.rsi[0]}, Price: {self.data_close[0]}, Lower Band: {self.bollinger.lines.bot[0]}" + Fore.RESET)
            self.order = self.buy()
            self.last_order_time = current_time
            self.last_order = 'buy'
        elif self.rsi > 70 and self.data_close > self.bollinger.lines.top and self.last_order != 'sell':
            self.log(Fore.LIGHTRED_EX + f"Sell signal with RSI: {self.rsi[0]}, Price: {self.data_close[0]}, Upper Band: {self.bollinger.lines.top[0]}" + Fore.RESET)
            self.order = self.sell()
            self.last_order_time = current_time
            self.last_order = 'sell'

    def notify_order(self, order):
        if order.status in [order.Submitted, order.Accepted]:
            return  # Pending order - do nothing

        if order.status in [order.Completed]:
            if order.isbuy():
                self.log(f'BUY EXECUTED, Price: {order.executed.price}, Cost: {order.executed.value}, Comm: {order.executed.comm}')
            elif order.issell():
                self.log(f'SELL EXECUTED, Price: {order.executed.price}, Cost: {order.executed.value}, Comm: {order.executed.comm}')
            self.bar_executed = len(self)
        elif order.status in [order.Canceled, order.Margin, order.Rejected]:
            self.log('Order Canceled/Margin/Rejected')

        self.order = None  # Reset orders

    def notify_trade(self, trade):
        if not trade.isclosed:
            return

        self.log(Fore.BLUE + f'OPERATION PROFIT, GROSS: {trade.pnl}, NET: {trade.pnlcomm}' + Fore.RESET)

# Set up the data and engine
cerebro = bt.Cerebro()
cerebro.addstrategy(RSI_Bollinger)

# Set initial conditions
cerebro.broker.set_cash(100_000)
cerebro.broker.setcommission(commission=0.001)

# Add observers and analyzers
cerebro.addobserver(bt.observers.Broker)
cerebro.addobserver(bt.observers.DrawDown)
cerebro.addanalyzer(bt.analyzers.SharpeRatio, _name='sharpe_ratio')
cerebro.addanalyzer(bt.analyzers.Transactions, _name='transactions')
cerebro.addanalyzer(bt.analyzers.TradeAnalyzer, _name='trade_analyzer')

cerebro.adddata(data)

# Run the strategy
results = cerebro.run()
strategy = results[0]

# Print results
print('Final Portfolio Value: %.2f' % cerebro.broker.getvalue())
print('Sharpe Ratio:', strategy.analyzers.sharpe_ratio.get_analysis())
print('Transactions:', strategy.analyzers.transactions.get_analysis())
print('Trade Analysis Results:', strategy.analyzers.trade_analyzer.get_analysis())


2024-06-10T04:29:00 - Buy signal with RSI: 29.887809324483413, Price: 436.5, Lower Band: 436.83078224814426
2024-06-10T04:30:00 - BUY EXECUTED, Price: 436.6499938964844, Cost: 436.6499938964844, Comm: 0.4366499938964844
2024-06-10T07:42:00 - Sell signal with RSI: 71.98700404907036, Price: 439.0, Upper Band: 438.7619088331895
2024-06-10T07:43:00 - SELL EXECUTED, Price: 438.8999938964844, Cost: 436.6499938964844, Comm: 0.4388999938964844
2024-06-10T07:43:00 - OPERATION PROFIT, GROSS: 2.25, NET: 1.3744500122070313
2024-06-10T08:54:00 - Buy signal with RSI: 26.80471012946633, Price: 437.95001220703125, Lower Band: 437.9861295744017
2024-06-10T08:55:00 - BUY EXECUTED, Price: 437.95001220703125, Cost: 437.95001220703125, Comm: 0.4379500122070313
2024-06-12T05:21:00 - Sell signal with RSI: 70.73997731379785, Price: 432.3999938964844, Upper Band: 432.3664596409787
2024-06-12T05:22:00 - SELL EXECUTED, Price: 432.3500061035156, Cost: 437.95001220703125, Comm: 0.43235000610351565
2024-06-12T05:22

In [40]:
import backtrader as bt
from datetime import datetime
import yfinance as yf

class IntradayTradingStrategy(bt.Strategy):
    params = (('rsi_length', 14), ('bollinger_length', 20), ('bollinger_dev', 2), ('macd1', 12), ('macd2', 26), ('macdsig', 9),)

    def __init__(self):
        self.data_close = self.datas[0].close
        self.rsi = bt.indicators.RSI(self.datas[0], period=self.params.rsi_length)
        self.bollinger = bt.indicators.BollingerBands(self.datas[0], period=self.params.bollinger_length, devfactor=self.params.bollinger_dev)
        self.macd = bt.indicators.MACD(self.datas[0], period_me1=self.params.macd1, period_me2=self.params.macd2, period_signal=self.params.macdsig)
        self.order = None  # To keep track of pending orders
        self.last_order_time = None  # To track the time of the last order

    def log(self, txt, dt=None):
        ''' Logging function for this strategy'''
        dt = dt or self.datas[0].datetime.datetime(0)
        print(f'{dt.isoformat()} - {txt}')

    def next(self):
        current_time = self.datas[0].datetime.datetime(0)

        # Check if 30 minutes have passed since the last order
        # if self.last_order_time and current_time < self.last_order_time + timedelta(minutes=30):
        #     return

        if self.order:
            return  # Skip if there's a pending order

        if self.rsi < 30 and self.data_close < self.bollinger.lines.bot and self.macd.macd > self.macd.signal:
            self.log(f"Buy signal with RSI: {self.rsi[0]}, Price: {self.data_close[0]}, Lower Band: {self.bollinger.lines.bot[0]}, MACD: {self.macd.macd[0]}, Signal: {self.macd.signal[0]}")
            self.order = self.buy()
            self.last_order_time = current_time
        elif self.rsi > 70 and self.data_close > self.bollinger.lines.top and self.macd.macd < self.macd.signal:
            self.log(f"Sell signal with RSI: {self.rsi[0]}, Price: {self.data_close[0]}, Upper Band: {self.bollinger.lines.top[0]}, MACD: {self.macd.macd[0]}, Signal: {self.macd.signal[0]}")
            self.order = self.sell()
            self.last_order_time = current_time

    def notify_order(self, order):
        if order.status in [order.Submitted, order.Accepted]:
            return  # Pending order - do nothing

        if order.status in [order.Completed]:
            if order.isbuy():
                self.log(f'BUY EXECUTED, Price: {order.executed.price}, Cost: {order.executed.value}, Comm: {order.executed.comm}')
            elif order.issell():
                self.log(f'SELL EXECUTED, Price: {order.executed.price}, Cost: {order.executed.value}, Comm: {order.executed.comm}')
            self.bar_executed = len(self)
        elif order.status in [order.Canceled, order.Margin, order.Rejected]:
            self.log('Order Canceled/Margin/Rejected')

        self.order = None  # Reset orders

    def notify_trade(self, trade):
        if not trade.isclosed:
            return

        self.log(f'OPERATION PROFIT, GROSS: {trade.pnl}, NET: {trade.pnlcomm}')

# Set up the data and engine
cerebro = bt.Cerebro()
cerebro.addstrategy(IntradayTradingStrategy)

# Set initial conditions
cerebro.broker.set_cash(100_000)
cerebro.broker.setcommission(commission=0.001)

# Add observers and analyzers
cerebro.addobserver(bt.observers.Broker)
cerebro.addobserver(bt.observers.DrawDown)
cerebro.addanalyzer(bt.analyzers.SharpeRatio, _name='sharpe_ratio')
cerebro.addanalyzer(bt.analyzers.Transactions, _name='transactions')
cerebro.addanalyzer(bt.analyzers.TradeAnalyzer, _name='trade_analyzer')

# Get data
data = bt.feeds.PandasData(dataname=yf.download('ITC.NS', interval='1m', period='5d'))
cerebro.adddata(data)

# Run the strategy
results = cerebro.run()
strategy = results[0]

# Print results
print('Final Portfolio Value: %.2f' % cerebro.broker.getvalue())
print('Sharpe Ratio:', strategy.analyzers.sharpe_ratio.get_analysis())
print('Transactions:', strategy.analyzers.transactions.get_analysis())
print('Trade Analysis Results:', strategy.analyzers.trade_analyzer.get_analysis())


[*********************100%%**********************]  1 of 1 completed


Final Portfolio Value: 100000.00
Sharpe Ratio: OrderedDict([('sharperatio', None)])
Transactions: OrderedDict()
Trade Analysis Results: AutoOrderedDict([('total', AutoOrderedDict([('total', 0)]))])


In [51]:
import requests
import backtrader as bt
from datetime import datetime, timedelta
import yfinance as yf
from colorama import Fore, Style

class RSI_Bollinger(bt.Strategy):
    params = (('rsi_length', 14), ('bollinger_length', 20), ('bollinger_dev', 2),)

    def __init__(self):
        self.data_close = self.datas[0].close
        self.rsi = bt.indicators.RSI(self.datas[0], period=self.params.rsi_length)
        self.bollinger = bt.indicators.BollingerBands(self.datas[0], period=self.params.bollinger_length, devfactor=self.params.bollinger_dev)
        self.order = None  # To keep track of pending orders
        self.last_order_time = None  # To track the time of the last order
        self.last_order = None  # To track the last order

    def log(self, txt, dt=None):
        ''' Logging function for this strategy'''
        dt = dt or self.datas[0].datetime.datetime(0)
        print(f'{dt.isoformat()} - {txt}')

    def send_signal(self, stock_name, strategy_name, signal, date, time, price, final_portfolio_value):
        url = 'http://127.0.0.1:8000/myapp/add_signal/'
        data = {
            'stock_name': stock_name,
            'strategy_name': strategy_name,
            'signal': signal,
            'date_time': f'{date} {time}',
            'price': price,
            'final_portfolio_value': final_portfolio_value
        }
        response = requests.post(url, data=data)
        if response.status_code == 200:
            print("Signal sent successfully!")
        else:
            print(f"Failed to send signal. Status code: {response.status_code}")

    def next(self):
        current_time = self.datas[0].datetime.datetime(0)
        
        # Check if 30 minutes have passed since the last order
        if self.last_order_time and current_time < self.last_order_time + timedelta(minutes=30):
            return

        if self.order:
            return  # Skip if there's a pending order

        if self.rsi < 30 and self.data_close < self.bollinger.lines.bot and self.last_order != 'buy':
            self.log(Fore.LIGHTGREEN_EX + f"Buy signal with RSI: {self.rsi[0]}, Price: {self.data_close[0]}, Lower Band: {self.bollinger.lines.bot[0]}" + Fore.RESET)
            self.order = self.buy()
            self.last_order_time = current_time
            self.last_order = 'buy'
            self.send_signal(
                stock_name='AAPL',
                strategy_name='RSI_Bollinger',
                signal='buy',
                date=current_time.date(),
                time=current_time.time(),
                price=self.data_close[0],
                final_portfolio_value=self.broker.getvalue()
            )
        elif self.rsi > 70 and self.data_close > self.bollinger.lines.top and self.last_order != 'sell':
            self.log(Fore.LIGHTRED_EX + f"Sell signal with RSI: {self.rsi[0]}, Price: {self.data_close[0]}, Upper Band: {self.bollinger.lines.top[0]}" + Fore.RESET)
            self.order = self.sell()
            self.last_order_time = current_time
            self.last_order = 'sell'
            self.send_signal(
                stock_name='AAPL',
                strategy_name='RSI_Bollinger',
                signal='sell',
                date=current_time.date(),
                time=current_time.time(),
                price=self.data_close[0],
                final_portfolio_value=self.broker.getvalue()
            )

    def notify_order(self, order):
        if order.status in [order.Submitted, order.Accepted]:
            return  # Pending order - do nothing

        if order.status in [order.Completed]:
            if order.isbuy():
                self.log(f'BUY EXECUTED, Price: {order.executed.price}, Cost: {order.executed.value}, Comm: {order.executed.comm}')
            elif order.issell():
                self.log(f'SELL EXECUTED, Price: {order.executed.price}, Cost: {order.executed.value}, Comm: {order.executed.comm}')
            self.bar_executed = len(self)
        elif order.status in [order.Canceled, order.Margin, order.Rejected]:
            self.log('Order Canceled/Margin/Rejected')

        self.order = None  # Reset orders

    def notify_trade(self, trade):
        if not trade.isclosed:
            return

        self.log(Fore.BLUE + f'OPERATION PROFIT, GROSS: {trade.pnl}, NET: {trade.pnlcomm}' + Fore.RESET)

# Set up the data and engine
cerebro = bt.Cerebro()
cerebro.addstrategy(RSI_Bollinger)

# Set initial conditions
cerebro.broker.set_cash(100_000)
cerebro.broker.setcommission(commission=0.001)

# Add data to cerebro
cerebro.adddata(data)

# Add observers and analyzers
cerebro.addobserver(bt.observers.Broker)
cerebro.addobserver(bt.observers.DrawDown)
cerebro.addanalyzer(bt.analyzers.SharpeRatio, _name='sharpe_ratio')
cerebro.addanalyzer(bt.analyzers.Transactions, _name='transactions')
cerebro.addanalyzer(bt.analyzers.TradeAnalyzer, _name='trade_analyzer')

# Run the strategy
results = cerebro.run()
strategy = results[0]

# Print results
print('Final Portfolio Value: %.2f' % cerebro.broker.getvalue())
print('Sharpe Ratio:', strategy.analyzers.sharpe_ratio.get_analysis())
print('Transactions:', strategy.analyzers.transactions.get_analysis())
print('Trade Analysis Results:', strategy.analyzers.trade_analyzer.get_analysis())


2024-06-10T04:20:00 - Buy signal with RSI: 29.481777567563583, Price: 12755.849609375, Lower Band: 12757.417677747
Failed to send signal. Status code: 500
2024-06-10T04:21:00 - BUY EXECUTED, Price: 12755.7998046875, Cost: 12755.7998046875, Comm: 12.7557998046875
2024-06-10T08:33:00 - Sell signal with RSI: 70.27545214186384, Price: 12686.0, Upper Band: 12679.613648858938
Failed to send signal. Status code: 500
2024-06-10T08:34:00 - SELL EXECUTED, Price: 12686.0, Cost: 12755.7998046875, Comm: 12.686
2024-06-10T08:34:00 - OPERATION PROFIT, GROSS: -69.7998046875, NET: -95.24160449218749
2024-06-11T07:04:00 - Buy signal with RSI: 29.03182834114473, Price: 12793.2998046875, Lower Band: 12804.784810901205
Failed to send signal. Status code: 500
2024-06-11T07:05:00 - BUY EXECUTED, Price: 12793.75, Cost: 12793.75, Comm: 12.793750000000001
2024-06-11T08:09:00 - Sell signal with RSI: 72.26524143044996, Price: 12909.150390625, Upper Band: 12904.48645006022
Failed to send signal. Status code: 500
2